In [2]:
### 필요한 module import 하기 ###
# 지난 실습에서 구현한 PrimeField 클래스와 Point 클래스 사용
from ECclass import *

In [3]:
### secp256k1 곡선의 유한체 파라미터
Prime = 2**256 - 2**32 - 977

In [4]:
### secp256k1 곡선을 위한 유한체 클래스 정의
class S256Field(PrimeField):

    # FieldElement 클래스를 상속하고, prime 만 secp256k1 의 prime 으로 할당
    def __init__(self, num, prime=None):
        super().__init__(num=num, prime=Prime)

    # 유한체 원소 표현시 일관되게 256 비트임을 표현하기 위해 zfill 메서드를 이용하여 빈자리를 '0' 으로 채움
    # 1 바이트 : 2개의 16진수로 표현 => 32 바이트 => 64 개 16진수로 표현
    def __repr__(self):
        #{:x}'.format(self.num) => num 을 16진수 문자열로 표현
        # zfill 메서드 : 파이썬 문자열 앞을 0으로 채우기
        return '{:x}'.format(self.num).zfill(64)

In [6]:
a = S256Field(7)
a

0000000000000000000000000000000000000000000000000000000000000007

In [7]:
### secp256k1 타원곡선 (y^2 = x^3 + Ax + B)
A = 0
B = 7
### secp256k1 타원곡선 Group generator 위수 (generator는 클래스 정의 후 point 설정)
N = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141

In [11]:
class S256Point(Point):

    # Point 클래스를 상속. S256Field 클래스를 이용하여 field 객체를 재정의.
    def __init__(self, x, y, a=None, b=None):
        a, b = S256Field(A), S256Field(B)
        # 만약 point 좌표가 S256Field 객체가 아닌 정수로 들어온 경우, 
        # 입력 정수에 대해 S256Field 객체를 생성하고 그에 따라 Point 객체 생성
        if type(x) == int:
            super().__init__(x=S256Field(x), y=S256Field(y), a=a, b=b)
        else:
            super().__init__(x=x, y=y, a=a, b=b)

    # SECPK256K1 곡선위의 Point 표현
    def __repr__(self):
        if self.x is None:
            return 'S256Point(infinity)'
        else:
            return 'S256Point({}, {})'.format(self.x, self.y)

    # 생성자의 위수가 N => r * G = (r mod N) * G
    def __rmul__(self, coefficient):
        coef = coefficient % N
        return super().__rmul__(coef)

In [13]:
### secp256k1 타원곡선 Group generator
G = S256Point(
    0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798,
    0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8)
G

S256Point(79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798, 483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8)

### ECDSA 구현에 필요한 모듈 및 함수 import

In [14]:
# 표준 해시함수 SHA256 사용을 위해 hashlib 모듈 import
# 의사난수생성을 위해 random 모듈의 randint 함수 import (randint(a,b): a 이상 b 이하의 random 정수를 반환)
import hashlib
from random import randint

In [15]:
### hash256 함수 정의
def hash256(s):
    return hashlib.sha256(hashlib.sha256(s).digest()).digest()

### ECDSA 공개키/개인키 생성

In [16]:
### 개인키 (서명키) & 공개키 (서명 검증키) 생성
# 개인키 prikey: 0 <= a  <= N-1
a = randint(0,N-1)
print('{:x}'.format(a).zfill(64))
# 공개키 pubkey: aG
Q = a*G
print(Q)

ae4e28308d01c77dd329318f3997b9c518799038b99ca21f534467d30fd0312b
S256Point(e6cfb10932b883417efed0f6e1c4a176cb47406fb71f8e740e183bd7e4956d52, 828b481fcae4cacc41224f635df5b68b9d0bfa6116751ae9d688233a2ed92d0f)


### ECDSA 서명 생성: 메시지 m 에 대해 서명 (x,s) 생성

In [17]:
### 서명한 메시지에 대한 해시값 계산
# 예시 메시지 m = 'my message' 에 대한 해시값 0 <= e(=f(m)) < 계산하기
# 1) 문자열 m 을 바이트열로 인코딩하여 저장 (각 문자를 아스키코드값으로 저장하는 바이트열로 변환)
m = b'my message'
print(m)
# 2) 바이트열 f(m) = sha256(sha256(m)) 계산하고, 정수형 데이터로 변환
# int.from_bytes : 
e = int.from_bytes(hash256(m),'big') % N
print('{:x}'.format(e).zfill(64))

b'my message'
0231c6f3d980a6b0fb7152f85cee7eb52bf92433d9919b9c5218cb08e79cce78


In [18]:
### 서명의 x 값 생성하기
# 1) random 정수 1 <= r < N 에 대해, R = rG 계산
r = randint(1,N-1)
R = r*G
print(R)
# 2) x 를 R 의 x 좌표 값으로 반환
x = R.x.num
print('{:x}'.format(x).zfill(64))

S256Point(2826e6704c2b0e673aba57a665826f2d1d1a6e50020d82e7ab27341245a80307, eeae3854931014215ed56ac57afc189e00d9c35b3b3933135fdca18cd190605d)
2826e6704c2b0e673aba57a665826f2d1d1a6e50020d82e7ab27341245a80307


In [20]:
### 서명의 s 값 생성하기
# 1) r 의 mod N 역원 계산
r_inv = pow(r,N-2,N)
# 2) s := rinv*(e+a*x) (mon N) 계산
s = (r_inv * (e + a*x)) % N
print('{:x}'.format(s).zfill(64))

165220e66016445829a0942828dced6bca69a1340ece1430cfd6af3ebb230c30


### ECDSA 서명 검증: 수신 메시지 m 에 대해 서명 (x,s) 검증

In [21]:
### 수신 메시지에 대한 해시값 계산
# 수신 메시지 M = 'my message' 에 대한 해시값 0 <= E(=f(M)) < 계산하기
# 1) 문자열 m 을 바이트열로 인코딩하여 저장 (각 문자를 아스키코드값으로 저장하는 바이트열로 변환)
M = b'my message'
# 2) 바이트열 f(m) = sha256(sha256(m)) 계산하고, 정수형 데이터로 변환
E = int.from_bytes(hash256(M),'big') % N
print('{:x}'.format(E).zfill(64))

0231c6f3d980a6b0fb7152f85cee7eb52bf92433d9919b9c5218cb08e79cce78


In [22]:
### v1 = E/s =(mod N) 계산
s_inv = pow(s,N-2,N)
v1 = (E * s_inv) % N
print('{:x}'.format(v1).zfill(64))

f2003220191bb2b03b0efcd257c9da20b078d0b7eb2d6d87fe766e2f6d8e2ca7


In [23]:
### v2 = x/s =(mod N) 계산
v2 = (x * s_inv) % N
print('{:x}'.format(v2).zfill(64))

d90a88e96dada30fcbdd949414c7655b9539c0f562583a939f596da4e80e7ca5


In [24]:
### V = v1*G + v2*Q 계산
V = v1*G + v2*Q
print(V)

S256Point(2826e6704c2b0e673aba57a665826f2d1d1a6e50020d82e7ab27341245a80307, eeae3854931014215ed56ac57afc189e00d9c35b3b3933135fdca18cd190605d)


In [25]:
### 서명의 x 와 V point 의 x 좌표가 같은지 검증
x == V.x.num

True